# Sweeps notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pprint
import os
import re
import json
import sys
import wandb
from model import *
from pytorch_lightning.loggers import WandbLogger
import tensorflow

c:\Users\Jesli's Laptop\AppData\Local\Programs\Python\Python39\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
# Define sweep config
sweep_config = {
    'method': 'random' # random, grid, or bayes
    }

In [3]:
# Define metric
metric = {
    'name': 'val_loss',
    'goal': 'minimize'   # minimize or maximize
    }
# add in sweep_config
sweep_config['metric'] = metric

In [4]:
epochs = 50
count = 1
architecture = 'CNN'
dataset = 'Poems'
# Define hyperparameter space
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'embedding_dim': {
        'values': [128, 256, 512]
        },
    }
# we can indicate the distribution for continuous variables
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.0001,
        'max': 0.1
      }
    })
# we set values that we want to track but don't want to change, just indicate 1 value
parameters_dict.update({
    'epochs': {'value': epochs},
    "architecture":{'value': architecture},
    "dataset": {'value': dataset},        
    })
# add params in sweep_config
sweep_config['parameters'] = parameters_dict

In [5]:
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'parameters': {'architecture': {'value': 'CNN'},
                'dataset': {'value': 'Poems'},
                'embedding_dim': {'values': [128, 256, 512]},
                'epochs': {'value': 50},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0.0001},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [6]:
# Log in to weights and biases
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jjdsantiago3 (msds_mlops2023_lt2). Use `wandb login --relogin` to force relogin


True

In [7]:
sweep_id = wandb.sweep(sweep_config, project="mlo-final-project")

Create sweep with ID: gs0lcwy2
Sweep URL: https://wandb.ai/msds_mlops2023_lt2/mlo-final-project/sweeps/gs0lcwy2


In [8]:
# define hyperparam search function
def hyperparam_search(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        # use config params
        learning_rate = config.learning_rate
        optimizer = config.optimizer
        embedding_dim = config.embedding_dim
        epochs = config.epochs

        # define model
        _model = poem_classifier_model()

        _model.load_data()
        _model.preprocess()
        _model.train(embd_dim=embedding_dim, epochs=epochs, lr=learning_rate, optimizer=optimizer)

        # print(_model.trained_model.history)

        for i in range(epochs):
            wandb.log({"train_loss": _model.trained_model.history['loss'][i],
                       "train_acc": _model.trained_model.history['acc'][i],
                       "val_loss": _model.trained_model.history['val_loss'][i], 
                       "val_acc": _model.trained_model.history['val_acc'][i]})
        
        results = _model.test()
        wandb.log({"test_loss": results[0], "test_acc": results[1]})

wandb.agent(sweep_id, hyperparam_search, count=count) # count - num iters

wandb: Agent Starting Run: 5i94635y with config:
wandb: 	architecture: CNN
wandb: 	dataset: Poems
wandb: 	embedding_dim: 256
wandb: 	epochs: 50
wandb: 	learning_rate: 0.06392504300409665
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:51: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('shade.When', 'shade. When')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:52: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now', 'afraid. Now')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:54: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Poem'].str.replace('afraid.Now,', 'afraid. Now,')
c:\Users\Jesli's Laptop\Desktop\Acads\MLO\mlo-final-project\code\model.py:55: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df_train['Poem'] = self.df_train['Po

Epoch 1/50
21/21 [==============================] - 1s 18ms/step - loss: 1.4443 - acc: 0.3214 - val_loss: 1.3046 - val_acc: 0.3988
Epoch 2/50
21/21 [==============================] - 0s 12ms/step - loss: 1.0744 - acc: 0.5441 - val_loss: 1.4146 - val_acc: 0.4286
Epoch 3/50
21/21 [==============================] - 0s 11ms/step - loss: 0.8689 - acc: 0.6607 - val_loss: 2.0865 - val_acc: 0.3631
Epoch 4/50
21/21 [==============================] - 0s 11ms/step - loss: 0.8971 - acc: 0.6457 - val_loss: 1.9746 - val_acc: 0.4107
Epoch 5/50
21/21 [==============================] - 0s 11ms/step - loss: 0.7432 - acc: 0.7294 - val_loss: 2.2097 - val_acc: 0.4167
Epoch 6/50
21/21 [==============================] - 0s 11ms/step - loss: 0.6591 - acc: 0.7638 - val_loss: 2.3688 - val_acc: 0.3869
Epoch 7/50
21/21 [==============================] - 0s 11ms/step - loss: 0.5927 - acc: 0.7758 - val_loss: 2.9101 - val_acc: 0.3631
Epoch 8/50
21/21 [==============================] - 0s 11ms/step - loss: 0.6694 - a

test_acc,▁
test_loss,▁
train_acc,▁▄▅▅▆▆▆▆▇▇▇▇▇██▇█▇████▇▇████▇███████▇███
train_loss,█▆▄▄▃▂▃▃▂▂▂▂▂▁▁▂▁▂▂▁▂▂▂▃▂▁▂▂▃▂▁▂▁▂▁▂▂▁▂▁
val_acc,▆█▄▇▆▄▄▄▃▆▄▅▄▃▃▃▃▄▃▃▄▂▃▄▄▄▂▃▃▄▃▃▂▁▁▁▂▂▂▄
val_loss,▁▁▂▂▂▃▃▃▃▄▄▅▄▄▄▅▅▅▅▅▆▅▆▆▆▆▆▆▇▇▇█▇▇▇▇▇▇▇█
test_acc,0.18667
test_loss,10.90408
train_acc,0.89836
train_loss,0.40985
val_acc,0.35714


In [9]:
wandb.finish()